In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append('../')

In [2]:
from generate_csv.single_session_data_extraction import *
from generate_features.emotional_rational_features import * # emo_ratio_single_session_tl_features, emo_ratio_single_session_hit_features

In [162]:
path = "/Users/smuel/Documents/Louie7/Developer/TenenGroup/WebTest/web_test3.txt"

In [163]:
raw = json2dict(path)

In [164]:
tst = pd.DataFrame.from_dict(raw, orient='index')
tst = tst.transpose()

In [165]:
df_tl = single_session_tl_df(raw)
df_tl.columns

Index(['visitNumber', 'visitId', 'visitStartTime', 'fullVisitorId', 'clientId',
       'channelGrouping', 'socialEngagementType', 'totals.visits',
       'totals.hits', 'totals.pageviews', 'totals.timeOnSite',
       'totals.sessionQualityDim', 'trafficSource.referralPath',
       'trafficSource.campaign', 'trafficSource.source',
       'trafficSource.medium', 'trafficSource.keyword',
       'trafficSource.adContent', 'trafficSource.adwordsClickInfo.campaignId',
       'trafficSource.adwordsClickInfo.adGroupId',
       'trafficSource.adwordsClickInfo.creativeId',
       'trafficSource.adwordsClickInfo.criteriaId',
       'trafficSource.adwordsClickInfo.page',
       'trafficSource.adwordsClickInfo.slot',
       'trafficSource.adwordsClickInfo.criteriaParameters',
       'trafficSource.adwordsClickInfo.gclId',
       'trafficSource.adwordsClickInfo.customerId',
       'trafficSource.adwordsClickInfo.adNetworkType',
       'trafficSource.adwordsClickInfo.isVideoAd', 'device.browser',
   

In [166]:
df_tl.head(1)

,visitNumber,visitId,visitStartTime,fullVisitorId,clientId,channelGrouping,socialEngagementType,totals.visits,totals.hits,totals.pageviews,...,geoNetwork.region,geoNetwork.metro,geoNetwork.city,geoNetwork.cityId,geoNetwork.networkDomain,geoNetwork.latitude,geoNetwork.longitude,geoNetwork.networkLocation,privacyInfo.analytics_storage,privacyInfo.ads_storage
0,5,1634415095,2021-10-16 21:17:52,3616516989199272664,842035978.1633897176,Generic Paid Search,Not Socially Engaged,1,134,14,...,County Tipperary,0,0,0,0,0.0000,0.0000,0,UNKNOWN,UNKNOWN


In [167]:
df_tl['totals.timeOnSite']

0    2455
Name: totals.timeOnSite, dtype: object

In [168]:
df_hit = single_session_hits_df(raw)
df_hit.shape

(134, 56)

In [169]:
session_time = np.sum(df_hit['hits.deltaTimeMS']) / 1000.0
session_time

2455.412

In [170]:
numeric_cols = ['totals.hits', 'totals.pageviews']
user , X = emo_ratio_single_session_tl_features(df_tl, session_time, numeric_cols)

In [171]:
X.head(1)

,trafficSourceRatio,trafficSourceSocialRatio,trafficSourceSeRatio,mediumSourceFbRatio,mediumSourceCpcRatio,mediumSourceOrganicRatio,avg_hits,avg_page_views,avg_time_on_site,end_of_month_ratio
0,0.0,0.0,0.0,0.0,0.0,0.0,134,14,2455.412,1


In [172]:
numeric_cols = ['numOfProducts','hits.eCommerceAction.action_type','hits.time']
Y = emo_ratio_single_session_hit_features(df_hit, numeric_cols)

In [173]:
Y.head(1)

,hits.products_per_session,hits.avg_viewd_product,hits.avg_time2hit,hits.avg_time_add_prod,hits.avg_time_rmv_prod,hits.support_page,hits.article_page,hits.search_keyword,hits.product_page,hits.category_page
0,1.126866,5,18323.970149,1493652.2,1812863.0,0,0,0,64,22


In [174]:
Z = pd.concat([X,Y], axis=1)

In [178]:
Z.head()

,trafficSourceRatio,trafficSourceSocialRatio,trafficSourceSeRatio,mediumSourceFbRatio,mediumSourceCpcRatio,mediumSourceOrganicRatio,avg_hits,avg_page_views,avg_time_on_site,end_of_month_ratio,hits.products_per_session,hits.avg_viewd_product,hits.avg_time2hit,hits.avg_time_add_prod,hits.avg_time_rmv_prod,hits.support_page,hits.article_page,hits.search_keyword,hits.product_page,hits.category_page
0,0.0,0.0,0.0,0.0,0.0,0.0,134,14,2455.412,1,1.126866,5,18323.970149,1493652.2,1812863.0,0,0,0,64,22


In [94]:
import pickle

In [176]:
model = pickle.load(open('../models/models_emo_ratio_model.pkl','rb'))

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/base.py:310: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


In [177]:
lbl = "active" if model.predict(Z.values)[0] else "non-active"
lbl

'active'

In [180]:
Z.avg_page_views[0]

14